In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding,Bidirectional,GRU, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

In [2]:
data=pd.read_csv("../input/twitter-and-reddit-sentimental-analysis-dataset/Twitter_Data.csv")
data.clean_text=data.clean_text.astype(str)



data = data[['clean_text','category']]

In [3]:
data=data[data.category !=0]
data['clean_text']=data['clean_text'].apply(lambda x: x.lower())
data['clean_text'] = data['clean_text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')
    
max_features = 2000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(data['clean_text'].values)
X = tokenizer.texts_to_sequences(data['clean_text'].values)
X = pad_sequences(X)

print(data[ data['category'] == 1].size)
print(data[ data['category'] == -1].size)

144500
71020


In [4]:
embed_dim = 128
gru_out = 196

model = Sequential()
model.add(Embedding(max_features, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(Bidirectional(GRU(gru_out, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

2021-11-13 15:34:10.343445: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-13 15:34:10.451210: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-13 15:34:10.451943: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-13 15:34:10.453166: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 47, 128)           256000    
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 47, 128)           0         
_________________________________________________________________
bidirectional (Bidirectional (None, 392)               383376    
_________________________________________________________________
dense (Dense)                (None, 2)                 786       
Total params: 640,162
Trainable params: 640,162
Non-trainable params: 0
_________________________________________________________________
None


In [5]:
Y = pd.get_dummies(data['category']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.3, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(75436, 47) (75436, 2)
(32331, 47) (32331, 2)


In [6]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 2, batch_size=batch_size, verbose = 2)

Epoch 1/2


2021-11-13 15:34:13.387595: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


2358/2358 - 912s - loss: 0.2932 - accuracy: 0.8731
Epoch 2/2
2358/2358 - 909s - loss: 0.2038 - accuracy: 0.9217


In [7]:
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

964/964 - 27s - loss: 0.1926 - accuracy: 0.9266
score: 0.19
acc: 0.93


In [8]:
text = ['Aditya is a very good boy']
text = tokenizer.texts_to_sequences(text)
text = pad_sequences(text, maxlen=28, dtype='int32', value=0)
print(text)
sentiment = model.predict(text,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0   76   55 1737]]
1/1 - 0s
positive
